In [1]:
#Importa as bibliotecas
from pymatgen.core.composition import *
import numpy as np
import pandas as pd
import ase.db
import json
import re

In [2]:
#Lê o arquivo para formar o dataframe das propriedades atômicas
df_atoms = pd.read_csv('Schleder2019_AtomicTable.csv')
df_atoms

,Element,Z,Electronegativity,IonizationPotential,ElectronAffinity,HOMO,LUMO,r_s_orbital,r_p_orbital,r_d_orbital,r_atomic_nonbonded,r_valence_lastorbital,r_covalent,Valence,PeriodicColumn,PeriodicColumn_upto18,NumberUnfilledOrbitals,Polarizability
0,H,1,2.20,-12.6833,-1.5273,-6.4925,0.7250,0.3865,NaN,NaN,0.37,0.3865,0.31,1.0,1.0,1.0,1.0,4.507107
1,He,2,NaN,-26.7499,3.0204,-15.7610,1.5714,0.2964,1.0292,0.4176,0.32,0.2964,0.28,2.0,8.0,18.0,0.0,1.383746
2,Li,3,0.98,-5.3606,-0.5863,-2.8744,-0.9074,1.6578,1.8874,2.0869,1.34,1.6578,1.28,1.0,1.0,1.0,1.0,164.000000
3,Be,4,1.57,-9.5007,0.7972,-5.6097,-2.0104,1.0805,1.2128,1.9594,0.90,1.0805,0.96,2.0,2.0,2.0,0.0,37.710000
4,B,5,2.04,-8.1261,0.0312,-3.6067,2.4547,0.8025,0.8348,1.3619,0.82,0.8348,0.84,3.0,3.0,13.0,5.0,20.530000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Fl,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.590000
114,Mc,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,Lv,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,Ts,117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Propriedades
prop = ['Z',
        'Electronegativity',
        'IonizationPotential',
        'ElectronAffinity',
        'HOMO',
        'LUMO',
        'r_s_orbital',
        'r_p_orbital',
        'r_d_orbital',
        'r_atomic_nonbonded',
        'r_valence_lastorbital',
        'r_covalent',
        'Valence',
        'PeriodicColumn',
        'PeriodicColumn_upto18',
        'NumberUnfilledOrbitals',
        'Polarizability']

In [4]:
df_atoms.set_index('Element', inplace = True)
dicio = df_atoms.to_dict('index')

In [5]:
data = ase.db.connect('./c2db-2021-06-24.db')
rows = data.select(is_magnetic=False) #Seleciona materias não metálicos e não magnéticos

#Listas de propriedades
lista = []
pesos = []
stch = []

#Features estatísticas
media_interm = {}

#Lista que guarda todas as informações
lista_completa = []

for row in rows:
    try:
        comp = Composition(row.formula).as_dict()
        elem = list(comp.items())
        media_interm['Material'] = row.formula
        media_interm['Space group'] = row.spacegroup
        media_interm['Exciton Binding Energy'] = row.E_B
        media_interm['Stoichiometry'] = row.stoichiometry
        media_interm['Crystal Type'] = row.crystal_type
    
        for i in prop:
            #Lista de propriedades de cada átomo
            for m in range(0, len(elem)):
                lista.append(dicio[elem[m][0]][i])
                pesos.append(elem[m][1])
                if (len(elem)==2):
                    stch.append(row.stoichiometry)
        
            #Valor médio
            media_interm[f'media_{i}'] = np.mean(lista)

    
            #édia ponderada
            avg = np.average(lista,weights=pesos)
            media_interm[f'media_pon_{i}'] = avg
    
            #Valor máximo e mínimo
            max_prop = max(lista)
            min_prop = min(lista)
            media_interm[f'max_{i}'] = max_prop
            media_interm[f'min_{i}'] = min_prop
    
            #Desvio padrão em relação a média
            media_interm[f'desvio_{i}'] = np.std(lista)
    
            #Desvio padrão em relação a média ponderada
            sum_prop = 0
            for j in lista:
                sub2 = (j - avg)**2
                sum_prop = sum_prop + sub2
            media_interm[f'desvio_pon_{i}'] = np.sqrt(sum_prop/len(lista)) 
        
            lista.clear()
            pesos.clear()
        
        lista_completa.append(media_interm.copy())
    except:
        pass

print(set(stch))
    
print(len(lista_completa))
df = pd.DataFrame(lista_completa)
df

{'AB', 'AB2'}
333


,Material,Space group,Exciton Binding Energy,Stoichiometry,Crystal Type,media_Z,media_pon_Z,max_Z,min_Z,desvio_Z,...,max_NumberUnfilledOrbitals,min_NumberUnfilledOrbitals,desvio_NumberUnfilledOrbitals,desvio_pon_NumberUnfilledOrbitals,media_Polarizability,media_pon_Polarizability,max_Polarizability,min_Polarizability,desvio_Polarizability,desvio_pon_Polarizability
0,As4,Pmna,0.565799,A,A-53-h,33.000000,33.000000,33,33,0.000000,...,3.0,3.0,0.000000,0.000000,29.800000,29.800000,29.80,29.80,0.000000,0.000000
1,Ag2I2,P4/nmm,1.082458,AB,AB-129-bc,50.000000,50.000000,53,47,3.000000,...,1.0,1.0,0.000000,0.000000,43.550000,43.550000,52.50,34.60,8.950000,8.950000
2,Ag2Se2,Pm,1.091100,AB,AB-6-ab,40.500000,40.500000,47,34,6.500000,...,2.0,1.0,0.500000,0.500000,39.370000,39.370000,52.50,26.24,13.130000,13.130000
3,Al2Te2,P-6m2,0.799185,AB,AB-187-hi,32.500000,32.500000,52,13,19.500000,...,5.0,2.0,1.500000,1.500000,46.200000,46.200000,55.40,37.00,9.200000,9.200000
4,GaAs,P3m1,0.397637,AB,AB-156-bc,32.000000,32.000000,33,31,1.000000,...,5.0,3.0,1.000000,1.000000,40.600000,40.600000,51.40,29.80,10.800000,10.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,ISbSe,P3m1,0.472964,ABC,ABC-156-ac,46.000000,46.000000,53,34,8.524475,...,3.0,1.0,0.816497,0.816497,34.463333,34.463333,42.55,26.24,6.659231,6.659231
329,MoSTe,P3m1,0.508695,ABC,ABC-156-ac,36.666667,36.666667,52,16,15.173076,...,6.0,2.0,1.885618,1.885618,47.456667,47.456667,86.00,19.37,28.188599,28.188599
330,WSSe,P3m1,0.506804,ABC,ABC-156-ac,41.333333,41.333333,74,16,24.239545,...,6.0,2.0,1.885618,1.885618,40.203333,40.203333,75.00,19.37,24.764292,24.764292
331,ZrSTe,P3m1,0.471690,ABC,ABC-156-ac,36.000000,36.000000,52,16,14.966630,...,8.0,2.0,2.828427,2.828427,59.123333,59.123333,121.00,19.37,44.341445,44.341445


In [6]:
df.to_csv('dataset_regression_final_full.csv') #Coloca o dataframe em um arquivo csv
data_materials = pd.read_csv('dataset_regression_final_full.csv')

In [7]:
#Preparando o dataset para começar o modelo
data_materials.drop('Unnamed: 0', axis = 'columns', inplace = True)
data_materials.dropna(inplace = True)
data_materials.info()

<class 'pandas.core.frame.DataFrame'>
Index: 329 entries, 0 to 332
Columns: 107 entries, Material to desvio_pon_Polarizability
dtypes: float64(101), int64(2), object(4)
memory usage: 277.6+ KB


In [100]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r_2
from sklearn.metrics import mean_absolute_error as mae
import seaborn as sns

#Train-Test Split
y = data_materials['Exciton Binding Energy']
x = data_materials.drop(['Material', 'Space group','Exciton Binding Energy', 'Stoichiometry', 'Crystal Type'] , axis = 'columns')

#Criando os dados de teste e treino:
x_train, x_test, y_train, y_test = tts(x, y , test_size = 0.20, random_state = 42)

gb = GradientBoostingRegressor()
gb.fit(x_train, y_train)

#Predição de treino
y_pred_train = gb.predict(x_train)
train_error = mse(y_train, y_pred_train, squared = True)

#Predição de teste
y_pred_test = gb.predict(x_test)
test_error = mse(y_test, y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(y_test, y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(y_test, y_pred_test))

Erro do train set: 0.012938500385588945
Erro do test set: 0.06234880429378861
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.7940955439377627
MSE do modelo: 0.06234880429378861
MAE do modelo: 0.1909984825159541


In [101]:
#Trainando para o Boruta.
X_train_n = x_train.values
y_train_n = y_train.values.ravel()

In [102]:
from boruta import BorutaPy

boruta = BorutaPy( gb, n_estimators='auto', verbose=2, random_state=42 ).fit(X_train_n, y_train_n )
features_importance = x_train.columns[boruta.support_]

print("Features mais importantes:")
print(features_importance) 
cols_selected = boruta.support_.tolist()

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	30
Rejected: 	72
Iteration: 	9 / 100
Confirmed: 	7
Tentative: 	23
Rejected: 	72
Iteration: 	10 / 100
Confirmed: 	7
Tentative: 	23
Rejected: 	72
Iteration: 	11 / 100
Confirmed: 	7
Tentative: 	23
Rejected: 	72
Iteration: 	12 / 100
Confirmed: 	10
Tentative: 	15
Rejected: 	77
Iteration: 	13 / 100
Confirmed: 	10
Tentative: 	15
Rejected: 	77
Iteration: 	14 / 100
Confirmed: 	10
Tentative: 	15
Rejected: 	77
Iteration: 	15 / 100
Confirmed: 	10
Tentative: 	15
Rejected: 	77
Iteration: 	16 / 100
Confirmed: 	15
Tentative

In [103]:
most_important_features = data_materials[['Material', 'Space group','Exciton Binding Energy', 
       'Stoichiometry', 'Crystal Type', 'media_pon_Z', 'media_pon_Electronegativity',
       'desvio_Electronegativity', 'desvio_pon_Electronegativity',
       'desvio_pon_IonizationPotential', 'media_pon_ElectronAffinity',
       'media_pon_HOMO', 'desvio_pon_HOMO', 'media_pon_LUMO',
       'media_pon_r_p_orbital', 'media_pon_r_d_orbital',
       'media_pon_r_atomic_nonbonded', 'desvio_pon_r_atomic_nonbonded',
       'media_pon_Valence', 'media_pon_PeriodicColumn', 'min_PeriodicColumn',
       'desvio_pon_PeriodicColumn', 'media_pon_NumberUnfilledOrbitals',
       'max_NumberUnfilledOrbitals', 'desvio_Polarizability',
       'desvio_pon_Polarizability']].copy()

In [105]:
#Train-Test Split
Y = most_important_features['Exciton Binding Energy']
X = most_important_features.drop(['Material', 'Space group','Exciton Binding Energy', 'Stoichiometry', 'Crystal Type'] , axis = 'columns')

#Criando os dados de teste e treino:
X_train, X_test, Y_train, Y_test = tts(X, Y , test_size = 0.2, random_state = 42)

gb = GradientBoostingRegressor(learning_rate=0.03, max_depth=5, min_samples_split=10,
                          subsample=0.5)
gb.fit(X_train, Y_train)

#Predição de treino
Y_pred_train = gb.predict(X_train)
train_error = mse(Y_train, Y_pred_train, squared = True)

#Predição de teste
Y_pred_test = gb.predict(X_test)
test_error = mse(Y_test, Y_pred_test, squared = True)

print("Erro do train set:", train_error)
print("Erro do test set:", test_error)
print("------AVALIAÇÃO DO MODELO------")
print("R2 Score do modelo:", r_2(Y_test, Y_pred_test))
print("MSE do modelo:", test_error)
print("MAE do modelo:", mae(Y_test, Y_pred_test))

Erro do train set: 0.022504571941077644
Erro do test set: 0.06517268785411308
------AVALIAÇÃO DO MODELO------
R2 Score do modelo: 0.7847697803556433
MSE do modelo: 0.06517268785411308
MAE do modelo: 0.20485269623917438


In [79]:
from sklearn.ensemble import BaggingRegressor
bag_gb = BaggingRegressor(estimator=gb, random_state=42)
bag_gb.fit(x_train, y_train)

#Treino
y_pred_train_bag=bag_gb.predict(x_train)
#Teste
y_pred_test_bag=bag_gb.predict(x_test)

#MSE
print("Train MSE:",mse(y_train, y_pred_train_bag))
print("Test MSE:",mse(y_test, y_pred_test_bag))
print("R2 Score:", r_2(y_test, y_pred_test_bag))

Train MSE: 0.038959643299590806
Test MSE: 0.0659831098268964
R2 Score: 0.782093393897604
